In [2]:
d = {"key": "value", "key1": "value1"}

In [5]:
d.key

AttributeError: 'dict' object has no attribute 'key'

In [6]:
from box import ConfigBox

In [7]:
d2 = {"key": "value", "key1": "value1"}

In [11]:
d2 = ConfigBox(d2)
d2

ConfigBox({'key': 'value', 'key1': 'value1'})

In [ ]:
d2.key

'value'

#### download data from google drive

In [15]:
import gdown

url = "https://drive.google.com/file/d/1rUPO5fl7EeoE_oSRH--tiLY9fn-1Niso/view?usp=sharing"
id = url.split("/")[-2]
output = 'data.zip'
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1rUPO5fl7EeoE_oSRH--tiLY9fn-1Niso
To: c:\Users\aldoe\Documents\Projects_Python\CT-SCAN\research\data.zip
100%|██████████| 12.1M/12.1M [00:01<00:00, 11.3MB/s]


'data.zip'

In [21]:
import os
%pwd

'c:\\Users\\aldoe\\Documents\\Projects_Python\\CT-SCAN\\research'

In [22]:
os.chdir("../")

In [24]:
%pwd

'c:\\Users\\aldoe\\Documents\\Projects_Python\\CT-SCAN'

In [ ]:
## Update the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [26]:
# 
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
## Update the configuraion manager in src config
from ast import Param


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config
    

In [29]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [32]:
## Update the components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self) -> str:
        """Fetch data from a given URL"""
        try:
            dataset_url = self.config.source_URL
            zip_download_path = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to {zip_download_path}")
            
            output = 'artifacts/data_ingestion/data.zip'
            gdown.download(dataset_url, output, quiet=False, fuzzy=True)
            logger.info(f"Downloaded data from {dataset_url} to {zip_download_path}")
            
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [33]:
## Update the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-05 15:49:21,443: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-05 15:49:21,444: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-05 15:49:21,445: INFO: common: created directory at: artifacts]
[2025-05-05 15:49:21,446: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-05 15:49:21,447: INFO: 329266585: Downloading data from https://drive.google.com/file/d/1rUPO5fl7EeoE_oSRH--tiLY9fn-1Niso/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?id=1rUPO5fl7EeoE_oSRH--tiLY9fn-1Niso
To: c:\Users\aldoe\Documents\Projects_Python\CT-SCAN\artifacts\data_ingestion\data.zip
100%|██████████| 12.1M/12.1M [00:01<00:00, 11.3MB/s]

[2025-05-05 15:49:29,255: INFO: 329266585: Downloaded data from https://drive.google.com/file/d/1rUPO5fl7EeoE_oSRH--tiLY9fn-1Niso/view?usp=sharing to artifacts/data_ingestion/data.zip]
